<a href="https://colab.research.google.com/github/hufexv/Basic_python_code/blob/main/DeepLearning_Procession.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Deep_learning_Moudle_training

In [53]:
#Modules

import numpy as np
import torch
import torch.nn as nn



class FcModule(nn.Module):
  def __init__(self, input_nums, output_nums, func):
    super(FcModule, self).__init__()
    self.w = nn.Parameter(torch.empty(input_nums, output_nums))
    self.b = nn.Parameter(torch.empty(output_nums))
    self.func = func
    for i in self.parameters():
        nn.init.normal(i.data)
  def forward(self, x):
    node = x @ self.w + self.b
    output = self.func(node)

    return output

class CrossModule(nn.Module):
  def __init__(self, input_nums):
    super(CrossModule, self).__init__()
    assert input_nums // 2 != 0
    self.split_i = input_nums // 2

  def forward(self, x):
    r1, r2 = torch.split(x, self.split_i, dim=-1)
    result = r1*r2
    output = torch.concat([x, result], dim=1)

    return output

class Classification(nn.Module):
  def __init__(self, input_nums, output_nums):
    super(Classification, self).__init__()
    self.Layer = nn.Sequential(
        FcModule(input_nums, 64, func=nn.Sigmoid()),
        FcModule(64, 64, func=nn.Sigmoid()),
        CrossModule(64),
        nn.Sequential(
            nn.Linear(96, 64),
            nn.Sigmoid()
        ),
        FcModule(64, output_nums, func=nn.Identity())
    )
  def forward(self, x):
    output = self.Layer(x)

    return output


In [49]:
#data_loader
import numpy as np
import sklearn.datasets
from torch.utils.data import Dataset, DataLoader
import torch

class My_dataset(Dataset):
  def __init__(self, X, Y):
    super(My_dataset, self).__init__()
    self.X = X
    self.Y = Y
  def __len__(self):
    return len(self.X)
  def __getitem__(self, index):
    x, y = self.X[index], self.Y[index]
    return np.asarray(x, dtype=np.float32), np.asarray(y, dtype=np.int64)

def Data_load(X, Y, batch_size, shuffle):
  ds = My_dataset(X, Y)
  return DataLoader(
      dataset=ds,
      batch_size=batch_size,
      shuffle=shuffle,
      num_workers=0
  )


In [40]:
#early
class stop_early(object):
  def __init__(self, max_step):
    super(stop_early, self).__init__()
    self._stop = False
    self.best_value = -1
    self.step = 0
    self.max_step = max_step

  def is_stop(self):
    return self._stop

  def update(self, value):
    if(self.step > self.max_step | value > self.best_value):
      return True
    else:
      self.step += 1
      return False


In [54]:
#Whole_procession
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.datasets import load_digits


#1 load_data
data = load_digits(return_X_y=False)
X, y = data.data, data.target
input_nums, output_nums = len(data.feature_names), len(data.target_names)

#2 preprocession
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=14)
train_loader = Data_load(train_x, train_y, batch_size=8, shuffle=True)
test_loader = Data_load(test_x, test_y, batch_size=16, shuffle=False)

#3 train_data
net = Classification(input_nums=input_nums, output_nums=output_nums)
loss_fn = nn.CrossEntropyLoss()
opt_fn = optim.SGD(net.parameters(), lr=0.02)
early = stop_early(max_step=100)

epoches = 1000
for epoch in range(epoches):
  net.train()
  for i, (x,y) in enumerate(train_loader):
    #forward
    score = net(x)
    loss = loss_fn(score, y)
    #backward
    opt_fn.zero_grad()
    loss.backward()
    opt_fn.step()

    print(f"{epoch} {i} Loss {loss.item():.3f}")

  with torch.no_grad():
    net.eval()
    total_pre_y = []
    for i, (x,y) in enumerate(test_loader):
      score = net(x)
      pre_y = torch.argmax(score, dim=1).numpy()
      total_pre_y.extend(list(pre_y))
      loss = loss_fn(score, y)

    print(metrics.classification_report(test_y, total_pre_y))
    print(metrics.accuracy_score(test_y, total_pre_y))







<ipython-input-53-a210e171818d>:16: FutureWarning: `nn.init.normal` is now deprecated in favor of `nn.init.normal_`.
  nn.init.normal(i.data)


0 0 Loss 8.899
0 1 Loss 7.707
0 2 Loss 4.705
0 3 Loss 5.828
0 4 Loss 3.585
0 5 Loss 4.688
0 6 Loss 5.229
0 7 Loss 2.253
0 8 Loss 2.419
0 9 Loss 2.579
0 10 Loss 2.434
0 11 Loss 3.120
0 12 Loss 2.372
0 13 Loss 2.500
0 14 Loss 2.952
0 15 Loss 2.233
0 16 Loss 1.947
0 17 Loss 2.255
0 18 Loss 2.454
0 19 Loss 2.504
0 20 Loss 2.114
0 21 Loss 2.229
0 22 Loss 2.195
0 23 Loss 2.545
0 24 Loss 2.148
0 25 Loss 2.661
0 26 Loss 2.366
0 27 Loss 2.423
0 28 Loss 2.282
0 29 Loss 2.089
0 30 Loss 2.406
0 31 Loss 1.937
0 32 Loss 2.487
0 33 Loss 1.915
0 34 Loss 2.354
0 35 Loss 2.273
0 36 Loss 2.247
0 37 Loss 1.928
0 38 Loss 2.111
0 39 Loss 1.965
0 40 Loss 2.310
0 41 Loss 2.184
0 42 Loss 2.277
0 43 Loss 2.144
0 44 Loss 2.492
0 45 Loss 2.330
0 46 Loss 1.959
0 47 Loss 2.267
0 48 Loss 1.863
0 49 Loss 2.051
0 50 Loss 1.906
0 51 Loss 2.446
0 52 Loss 2.119
0 53 Loss 2.045
0 54 Loss 2.366
0 55 Loss 2.371
0 56 Loss 2.276
0 57 Loss 2.112
0 58 Loss 2.148
0 59 Loss 2.266
0 60 Loss 1.814
0 61 Loss 1.993
0 62 Loss 1.988
0 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Streaming output truncated to the last 5000 lines.
           2       0.93      0.93      0.93        29
           3       0.91      0.84      0.87        37
           4       0.97      0.94      0.95        31
           5       0.79      0.94      0.86        35
           6       0.90      0.97      0.94        37
           7       0.92      0.81      0.86        43
           8       0.83      0.79      0.81        38
           9       0.96      0.88      0.92        50

    accuracy                           0.90       360
   macro avg       0.90      0.90      0.90       360
weighted avg       0.90      0.90      0.90       360

0.8972222222222223
93 0 Loss 0.025
93 1 Loss 0.014
93 2 Loss 0.021
93 3 Loss 0.027
93 4 Loss 0.060
93 5 Loss 0.022
93 6 Loss 0.028
93 7 Loss 0.016
93 8 Loss 0.002
93 9 Loss 0.183
93 10 Loss 0.008
93 11 Loss 0.005
93 12 Loss 0.003
93 13 Loss 0.011
93 14 Loss 0.033
93 15 Loss 0.013
93 16 Loss 0.026
93 17 Loss 0.139
93 18 Loss 0.027
93 19 Loss 0.010
93 2

KeyboardInterrupt: 